In [3]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
from nltk.corpus import stopwords
import string
import os
import json
from nltk.corpus import wordnet
import glob
import csv
import pandas as pd

In [4]:
# read kwds list
def read_kwd(txtfile):
    f = open(txtfile)
    content = f.read()
    kw_list = (content.lower()).split('\n')
    return kw_list

In [ ]:
def get_synonym(voc_concept):
    #Creating a list
    synonyms = []
    for syn in wordnet.synsets(voc_concept,lang='fra'):#, lang='fra'
        for lm in syn.lemmas('fra'):
            synonyms.append(lm.name())#adding into synonyms
    synonyms = set(synonyms)
    return synonyms

In [ ]:
# data = ['incendie', 'eau']
def retriev_syno(voc_concept):
    data = read_kwd(voc_concept)
    root, filname = os.path.split(voc_concept)
    filname = filname.split('.txt')[0]
    synos = {}
    for t in data:
        synos[t] = []
        ts = t.split(' ')
        if ts.__len__() == 1:
            syn1 = [get_synonym(a) for a in ts]
            syn1[0] = [v for v in list(syn1[0]) if v!=t]
            synos[t] = list(syn1[0])
        if ts.__len__() > 1:
            syn = [get_synonym(a) for a in ts]
#             print(syn)
            for s in syn[0]:
                test = " ".join(ts[1:])
                w = s.lower() + ' ' + test
#                 print(w)
                if w != t:
                    synos[t].append(w)
#     print(synos)
    return synos

In [ ]:
expert_terms_nature_danger = './Herelles_ress/termes_graines_natural.dangers.txt'
expert_terms_urban = "./Herelles_ress/termes_graines_urbanisme.txt"
expert_terms_synos = retriev_syno(expert_terms_urban)

In [ ]:
import pandas as pd
# dic = {'a': ['ACI', 'BBY', 'AMRX'], 'b': ['A;BM', 'AG']} 
df = pd.DataFrame.from_dict(expert_terms_synos, orient='index').T #Convert dict to df
# print(df)
df

In [ ]:
df.to_csv("./Herelles_ress/termes_graines_urbanisme_extend.csv",header=True) #Convert df to csv

In [7]:
# List data to csv file
def write_list_to_txt(terms_list, outfile):
    """Write the list to csv file."""
    with open(outfile, 'w') as f:
        for item in terms_list:
            f.write("%s\n" % item)
    return outfile

In [8]:
# recoder pour prendre en compte un fichier csv, la sortie du vocabulaire de concepts final
def generate_new_terms(voc_concept):
    data = read_kwd(voc_concept)
    root, filname = os.path.split(voc_concept)
    filname = filname.split('.txt')[0]
    L = []
    for t in data:
        t = t.split(' ')
        if t.__len__() == 1:
            syn = [get_synonym(a) for a in t]
            for v in syn[0]:
                L.append(v)
        if t.__len__() > 1:
            syn = [get_synonym(a) for a in t]
            #print(syn)
            for s in syn[0]:
                test = " ".join(t[1:])
                w = s.lower() + ' ' + test
                #print('zzz', w)
                L.append(w)
    #print(L.__len__())
    L = set(L)
    #print(L.__len__())
#     write_list_to_txt(L,filname+'_syno.txt')
    return list(L)

In [11]:
# usage
data = './Herelles_ress/termes_graines_natural.dangers.txt'
# extended_data = generate_new_terms(data)